# Latent Dirichlet Allocation

### Megan Robertson and Anna Yanchenko


## I. Abstract

Latent Dirichlet Analysis (LDA) is a method used to model the generative process of creating discrete data such as text corpora. LDA can take a large amount of data and create descriptions of that data. This method reduces the size of the data to these short descriptions while still maintaining the relationships necessary to carry out various inferences.  For this project, we implemented the LDA algorithm presented in the paper *Latent Dirichlet Allocation*, written by David M. Blei, Andrew Y. Ng, and Michael I. Jordan. The implementation of the algorithm is used to analyze the text of some famous American historical documents and to create clusters of movies based on user rating data. 

## II. Background

LDA is most well-known for its application in the analysis of text data. It is used to create topics for documents, classify documents based on these topics and to determine which documents are similar to one another. LDA is based on the "bag-of-words" assumption, that is, that there is exchangeability between documents and words in the corpus. The algorithm can also be used in other problems that have a similar structure to the document generating method described above. For example, in Blei et.al, 2003, the authors used a data set where web site users provide ratings about movies they enjoy. In this example, the users are analogous to the documents and their preferred movies are “words”.  The model is used to predict what movies a user may like based on their previous ratings.

The algorithm assumes a generative process for the creation of each document **w** in a corpus D.  For each document in a corpus, the following generative process is assumed  (Blei et al., pg. 996):

1. Select number of words for the document, N $\sim Poission(\xi$)
2. Choose $\theta \sim Dirichilet(\alpha)$
3. For each word, $w_n$:
  
 a. Select topic, $z_n \sim Multinomial(\theta)$

 b. Select a word $w_n$ from $p(w_n |z_n, \beta)$

This process leads to the following posterior distribution:

$$ p(\theta, \textbf{z} | \textbf{w}, \alpha, \beta) = \dfrac{p(\theta, \textbf{z}, \textbf{w} | \alpha, \beta)}{p(\textbf{w}, \alpha, \beta)}$$

However, this posterior is intractable, and must be approximated. 

There are multiple alternative algorithms that can be used for text analysis. These include the term-frequency inverse-document-frequency (tf-idf) matrix, latent semantic indexing (LSI) and the probabilistic latent semantic indexing (pLSI) model. In Section 7 of the paper, the authors present the results of document modeling and document classification. In this section, the authors’ results show that their implementation of LDA performs better than competing methods in terms of perplexity measure, where better generalization performance is defined by a smaller perplexity measure. 

This section demonstrates that other methods are prone to overfitting. As the number of topic increases, some of the alternative algorithms induce words that have small probabilities. This occurs because the documents in the corpus are divided into more collections. This can result in the perplexity measure becoming very large for these alternative algorithms. The problem comes from the requirement that the topic proportions in a future document must be seen in at least one of the training documents. On the other hand, LDA does not have this overfitting problem. See Section 7 of Blei et al for more details.

A disadvantage of LDA is that exact inference of the posterior is impossible as a result of intractability. Thus, it is necessary for the user to implement an approximating technique such as variational inference, a Gibbs sampler, or another technique. The approximation of the posterior can be computationally intensive and time-consuming.  Additionally, for LDA, the number of topics $k$ must be decided before the algorithm is run.

## III. Implementation

As mentioned in the background, the posterior is intractable. Therefore it is necessary to approximate the posterior, and this is done using variational inference. This approximation is:

$$q(\theta, \textbf{z} | \gamma, \phi) = q(\theta | \phi) \prod_{n=1}^N q(z_n | \phi_n)$$

where $\gamma$ is a Dirichlet random variable and ($\phi_1, ..., \phi_N)$ are Multinomial.

These variables are used to approximate the posterior using "an alternating variational EM procedure that maximizes a lower bound with respect to the variational parameters $\gamma$ and $\phi$, and then, for fixed values of the variational parameters, maximizes the lower bound with respect to the model parameters $\alpha$ and $\beta$" (Blei et al, 1005).

The details of the derivations and equations can be found in the Appendix of the paper (Blei et al, 1018).

### a. Code

The LDA algorithm was implemented using the Python programming language in the Jupyter notebook environment. Descriptions for the functions used in the algorithm are included with the code below. 



### b.	Testing and Base Cases

In addition to the implementation described above, various checks have been included in the function to prevent incorrect arguments.  One check is that the number of topics specified by the user must be greater than one. It is not possible to have zero or less than zero topics, and one topic would just be described by the entire document. In addition, there is a check in place to ensure that the corpus is not empty. Lastly, there are checks to ensure that the tolerance is greater than zero and that the entry for needToSplit is either zero or one. These checks print messages that inform the user of why their input was problematic.

### c. Functions

The code included below is the optimized code for the project. For the unoptimized versions referenced later in the paper, see the Appendix, section A.1, also included in the Submission folder.

In [2]:
%reload_ext cython

In [3]:
! pip install stop_words
! pip install nltk

**Toy Example Corpus **
From HW5 Question 3

In [4]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4 

**Function to make Corpus Matrix** 


**make_word_matrix(corpus, needToSplit)**

Inputs: 
- *corpus*: a dictionary of text documents, each element of the dictionary is a separate document in string format
- *needToSplit*: if each document is one single string and not split into individual words, needToSplit = 1, otherwise,
    if the document is already split into individual words, needToSplit = 0
    
Output: list[c, wordOrder, M]
- *c*: a list of corpus matricies, one matrix for each document. Each row corresponds to each word in that document, while each colulmn corresponds to a unique word in the vocabulary (V possible words) for all documents.  Following the notation of Blei, et. al., the nth word in the vocabulary is represented by $w^n = 1$ and $w^m$ = 0 for all $m\neq n$
- *wordOrder*: a list of the unique words in the vocabulary for the corpus; these are the column labels for each matrix in c
- *M*: the number of documents in the corpus


In [5]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer



#Function to make document, word matricies for LDA#
def make_word_matrix(corpus, needToSplit):
    
    #define dictionary to store "cleaned" words
    cleanCorpus = {}
    #Define stop words
    stopWords = get_stop_words('english')
    
    
    #Initialize stemmer
    p_stemmer = PorterStemmer()

    
    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    
    #Check to make sure that dictionary isn't empty#
    if M ==0:
        print("Input dictionary is empty")
        return;
    
    removePunc = string.punctuation
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]

        #Remove stop words#
        text = [word for word in words if word not in stopWords]
        # stem tokens
        text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)
        cleanCorpus[i] = text
        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)
    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)
    
    #For each document in docs, caculate frequency of the words#
    for i in cleanCorpus:
        text = cleanCorpus[i]
        N = len(text)
        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            #Find which word in vocabulary current word in document corresponds to#
            v = wordOrder.index(word)
            #Add a 1 to that column in the wordsMat matrix#
            wordsMat[count, v] = 1
            count += 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

**Output for toy docs example**

In [6]:
corpusMatrix = make_word_matrix(docs, 1)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.

### Variational Inference

In [7]:
import numpy as np
import scipy
from scipy import special

**E-Step:** This function uses variational inference to perform the E step in the EM algorithm to estimate the paramteters in the model.  See page 1004 of Blei, et. al. for the derivation.

**Estep(k, d, alpha, beta, corpusMatrix, tol)**

**Inputs:**
- *k*: the number of topics assumed for the LDA model, an integer
- *d*: the current document in the corpus to perform the E step on, an integer
- *alpha*: the current alpha vector for the model, a k-dimensional vector
- *beta*: the current beta matrix for the model, a k x V matrix
- *corpusMatrix*: the output of make_word_matrix() from above
- *tol*: the convergence criteria for the values of phi and gamma found in the Estep

**Output: list(newPhi, gamma)**
- *newPhi*: the updated result for phi from the Estep, a N x k matrix (N is the number of words in document d)
- *gamma*: the updated result for gamma from the Estep, a k-dimensional vector

In [8]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        #Update phi
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma

        #Check convergence criteria
        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

### Parameter Estimation

**M Step:** In the E step above, we maximized a lower bound with respect to gamma and phi, and in the M step, for fixed values of these variational parameters, we maximize the lower bound of the log likelihood with repsect to alpha and beta to update these values (combined, these two steps give approximate empirical Bayes estimates for the LDA model).  See pg. 1006 and appendix A.2 in Blei et. al. for the derivation.  

The cy_alphaUpdate() function uses the linear Newton-Rhapson method to update the Dirichlet parameters, alpha, while the cy_betaUpdate() function maximizes for beta.

**cy_alphaUpdate(k, M, alphaOld, gamma, tol):**

**Inputs:**
- *k*: the number of topics, an integer
- *M*: the number of documents in the corpus, an integer
- *alphaOld*: the previous alpha value, a k-dimensional vector
- *gamma*: the value which maximized the lower bound in the Estep above, a k-dimensional vector
- *tol*:  the convergence criteria for the linear Newton-Rhapson method to update alpha

**Output**: alpaNew
- *alphaNew*: the updated version of alpha which maximizes the lower bound of the log likelihood, a k-dimensional vector



**cy_betaUpdate(k, M, phi_dense, gamma, c_1, c_2):**

**Inputs:**
- *k*: the number of topics, an integer
- *M*: the number of documents in the corpus, an integer
- *phi_dense*: a list of dense matricies that are the output from the Estep above run on each document, each matrix is max $N_i$ x k, where max $N_i$ is the maximum number of words in a single document in the corpus
- *c_1*: the number of words in each document in the corpus, a M-dimensional vector
- *c_2*: a list of dense matricies, the same as the c portion of the output of make_word_matrix above, but each matrix is now the same dimensions (with rows of zeros added as necessary), each matrix is max $N_i$ x V, where max $N_i$ is the maximum number of words in a single document in the corpus

**Outputs**: beta
- *beta*: the updated version of beta which maximizes the lower bound of the log likelihood, a k x V matrix

In [9]:
%%cython  -lgsl

import cython
import numpy as np
cimport numpy as np
import scipy
from cython_gsl cimport *

#Update alpha using linear Newton-Rhapson Method#

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:] cy_alphaUpdate(int k, int M, double[:] alphaOld, double[:, :] gamma, double tol):
    cdef double[:] h = np.zeros(k)
    cdef double[:] g = np.zeros(k)
    cdef double[:] alphaNew = np.zeros(k)

    cdef int converge
    cdef int i, d
    cdef double docSum
    cdef double[:] step = np.zeros(k)
    cdef double c, s1, s2

    cdef double alpha_sum = 0
    for i in range(k):
        alpha_sum += alphaOld[i]

    converge = 0
    while converge == 0:
        z = -gsl_sf_psi_n(1, alpha_sum)

        s1 = 0
        s2 = 1.0/z
        for i in range(0, k):
            docSum = 0
            for d in range(M):
                docSum += gsl_sf_psi(gamma[d,i]) - gsl_sf_psi(sum(gamma[d]))
            g[i] = M*(gsl_sf_psi(alpha_sum) - gsl_sf_psi(alphaOld[i])) + docSum
            h[i] = M*gsl_sf_psi_n(1, alphaOld[i])
            
            s1 += g[i]/h[i]
            s2 += 1.0/h[i]
        c = s1/s2

        for i in range(k):
            step[i] = (g[i] - c)/h[i]
        # step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            for i in range(k):
                alphaNew[i] = alphaOld[i] + step[i]
            alphaOld = alphaNew

    return alphaNew

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:, :] cy_betaUpdate(int k, int M, double[:, :, :] phi_dense,long[:] c_1, double[:, :, :] c_2):

    cdef int i, j, d, n
    cdef int Nd
    cdef double wordSum

    #Calculate beta#
    cdef int V = c_2[0].shape[1]
    cdef double[:, :] beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(M):
                Nd = c_1[d] # c[d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi_dense[d, n,i]*c_2[d, n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]

    return beta

### LDA Function: LDA(k, corpusMatrix, tol)

For each document d, the function runs the E step, then for all documents runs the M step update for alpha and beta.  This variational update and parameter estimation update continues until alpha or beta converges to within the specified tolerance.  The final values of phi, gamma, alpha and beta are returned for all M documents in a list.

**Inputs:**
- *k*: the number of topics, an integer
- *corpusMatrix*: the output of make_word_matrix() above for the desired corpus
- *tol*: the convergence criteria to be used in the E-step, alpha update in the M-step and the convergence to be used for the overall maximization 

**Output:** list([phi, gamma, alphaNew, betaNew])
- *phi*: a list of matricies, the final phi matrix values for each document in the corpus
- *gamma*: a list of vectors, the final gamma vector for each document in the corpus
- *alphaNew*: the final alpha vector, a k-dimensional vector
- *betaNew*: the final beta matrix, a k x V matrix


In [10]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, corpusMatrix, tol):

    # create rectangular matrices for c
    c = corpusMatrix[0]
    r_c = len(c)
    c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
    n_c = max(c_1)
    p_c = c[0].shape[1]
    c_2 = np.zeros((r_c, n_c, p_c))
    for i, j in enumerate(c_1):
        c_2[i, :j, :] = c[i]
    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = np.full(shape = k, fill_value = 50/k) + np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        
        ##E-step##
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        # create rectangular matrices for phi
        r = len(phi)
        phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
        n = max(phi_1)
        p = phi[0].shape[1]
        phi_2 = np.zeros((r, n, p))
        for i, j in enumerate(phi_1):
            phi_2[i, :j, :] = phi[i]

        ##M - step##
        #Update alpha and beta#     
        gamma = np.array(gamma)
        alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
        betaNew = np.array(cy_betaUpdate(k, M, phi_2, c_1, c_2))
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
       
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

**Time for toy example to run**

In [11]:
%%time
np.random.seed(97)
res = LDA(3, corpusMatrix, 0.1)

CPU times: user 4.76 ms, sys: 623 µs, total: 5.38 ms
Wall time: 5.85 ms


**Toy example LDA output**

In [12]:
phi_toy = res[0][0]
phi_toy

[array([[ 0.09689023,  0.5125238 ,  0.39058597],
        [ 0.10309406,  0.63093592,  0.26597002],
        [ 0.5518897 ,  0.17445205,  0.27365826]]),
 array([[ 0.3948353 ,  0.14007042,  0.46509427],
        [ 0.1557428 ,  0.20170723,  0.64254998],
        [ 0.24561568,  0.43189135,  0.32249297],
        [ 0.17846235,  0.63008699,  0.19145066],
        [ 0.17572041,  0.23097513,  0.59330447]]),
 array([[ 0.16226174,  0.6583813 ,  0.17935696],
        [ 0.40071343,  0.14074978,  0.45853679],
        [ 0.17916578,  0.23317472,  0.5876595 ]]),
 array([[ 0.11610625,  0.62060252,  0.26329123],
        [ 0.58413674,  0.16126668,  0.25459658],
        [ 0.72925797,  0.13686583,  0.1338762 ],
        [ 0.72925797,  0.13686583,  0.1338762 ],
        [ 0.72925797,  0.13686583,  0.1338762 ],
        [ 0.72925797,  0.13686583,  0.1338762 ]])]

In [13]:
gamma_toy = res[0][1]
gamma_toy

array([[ 18.63161569,  18.71214015,  17.96387036],
       [ 19.03011823,  19.0289595 ,  19.24854846],
       [ 18.62188265,  18.42653418,  18.25920936],
       [ 21.49701656,  18.72356092,  18.08704871]])

In [14]:
alpha_toy = res[0][2]
alpha_toy

array([ 17.98476059,  17.34928734,  17.03592818])

In [15]:
beta_toy = res[0][3]
beta_toy

array([[ 0.0350067 ,  0.12705064,  0.056676  ,  0.18142558,  0.46585553,
         0.02591351,  0.03922529,  0.02487242,  0.01547355,  0.02850078],
       [ 0.23550474,  0.05284256,  0.0873401 ,  0.06317293,  0.10301738,
         0.12388906,  0.08126995,  0.03795569,  0.09644273,  0.11856486],
       [ 0.09757672,  0.17028432,  0.21772724,  0.09739117,  0.0987278 ,
         0.03306697,  0.05945609,  0.11846308,  0.07200999,  0.03529661]])

**Function to Find Highest Probability Words for Each Topic: **
Run on toy example in (d) ii. below

**mostCommon(beta, wordList, p):**

**Inputs:**
- *beta*: the final value for the beta matrix from the LDA function above
- *wordList*: the list of the unique words in the vocabulary in the corpus that LDA was run on to find beta, the output of the make_word_matrix()[1]
- *p*: the number of words to return for each topic

**Output: list(topicWords)**
- *topicWords*: for each topic, the words in the corpus that have the highest probability of occuring under  that topic

In [16]:
#function to return the most probable words
#p is the number of words you want returned for each topic
def mostCommon(beta, wordList, p):
    k = beta.shape[0]
    topicWords = []
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    for i in range(0, k):
        document = betaDF.loc[i,:]
        document.sort(1, ascending = 0)
        mostCommon = pd.DataFrame(document[0:p])
        topicWords.append(mostCommon)
    return(topicWords)

### d.	Tests for Correctness

### *i.	Generative Model*

A generative model check was utilized in order to evaluate the functionality of the algorithm. A random number was generated in order to determine the number of documents from a Poisson(5) distribution, and the resulting number was six. Then, for each of the six documents, a random number of words for each document was drawn from a Poisson(5) distribution. These documents had between three and seven words. Random values were used to create a $\beta$ vector, and the $\alpha$ vector was set to be 50/k, where k is the number of topics and k = 3. The 50/k value was chosen based on the results shown in Gregor Heinrich’s *Parameter Estimation for Text Analysis*  (Heinrich, 2008, 24). This value was chosen such that $\alpha$ converges logically. With a random initialization, the algorithm returned unrealistic values for $\alpha$, and it would require much more time to converge.

The corpus was defined to have five unique words that are not stop words.
The set-up described in the previous paragraph was used to determine which of the words were in each of the documents. This process followed the generative model described in Blei et. al’s paper and above (Blei et al, 2003, 996). Once the documents were generated, our LDA implementation was used on this corpus to estimate the $\alpha$ and $\beta$ values. The following table displays the mean squared error (MSE) between the actual $\alpha$ and $\beta$ values used to create the corpus and the ones determined by our algorithm.

| Alpha MSE | Beta MSE |
|-----------|----------|
|      1.50 |     0.77 |

Thus, the values returned by the algorithm are relatively close to the ones used to generate the corpus. It is not surprising that the MSE for the $\beta$ vector is larger because there are more elements in the beta vector. The $\alpha$ vector converges faster than the $\beta$ vector, so this could be the reason for the larger MSE for $\beta$. 


In [17]:
#Generate data
np.random.seed(7)
#Draw number of documents
M = np.random.poisson(lam = 5)
k = 3  #Number of topics
alpha = np.full(shape = k, fill_value = 50/k) +np.random.rand(k)
V = 5  #Number of words in vocabulary
betaOld = np.random.rand(k, V)
betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
corpus = []
for i in range(0,M):
    #Draw number of words in document i
    N = np.random.poisson(5)
    theta = np.random.dirichlet(alpha)
    docWords = np.zeros(N)
    for j in range(0,N):
        #Draw latent topics for each word
        z = np.random.multinomial(1,theta)
        zn = list(z).index(1)
        #Draw each word given that latent topic
        w = np.random.multinomial(1,betaOld[zn,:])
        docWords[j] = list(w).index(1)
    corpus.append(docWords)
corpus

[array([ 4.,  1.,  3.,  1.,  0.]),
 array([ 4.,  3.,  1.]),
 array([ 3.,  1.,  2.,  1.,  1.,  4.,  0.]),
 array([ 4.,  4.,  1.,  0.,  1.]),
 array([ 4.,  2.,  1.,  2.,  3.,  1.,  0.]),
 array([ 2.,  1.,  1.,  4.,  3.])]

In [18]:
#Word choices apple, banana, grape, orange, watermelon
s1 = "Watermelon banana orange banana apple"
s2 = "Watermelon orange banana"
s3 = "Orange banana grape banana banana watermelon apple."
s4 = "Watermelon watermelon banana apple banana"
s5 = "Watermelon grape banana grape orange banana apple"
s6 = "Grape banana banana watermelon orange"
docsTest = collections.OrderedDict()
docsTest["s1"] = s1
docsTest["s2"] = s2
docsTest["s3"] = s3
docsTest["s4"] = s4 
docsTest["s5"] = s5 
docsTest["s6"] = s6 

In [19]:
alpha

array([ 16.7387178 ,  16.93510565,  17.16654917])

In [20]:
betaOld

array([[ 0.30623643,  0.36237236,  0.17175045,  0.02972794,  0.12991282],
       [ 0.35935154,  0.08430178,  0.17861983,  0.36788995,  0.00983689],
       [ 0.18543475,  0.29337665,  0.07111187,  0.16936021,  0.28071651]])

In [21]:
corpusMatTest = make_word_matrix(docsTest, 1)
np.random.seed(1)
phi, gamma, alphaNew, betaNew =  LDA(3, corpusMatTest, 0.1)[0]
alphaNew

array([ 17.24430799,  17.92955043,  16.16631559])

In [22]:
betaNew

array([[ 0.11557991,  0.41398094,  0.05114086,  0.11955674,  0.29974155],
       [ 0.05963179,  0.6014481 ,  0.09408642,  0.17555618,  0.06927752],
       [ 0.21675927,  0.04987748,  0.24766886,  0.17423878,  0.31145561]])

**Alpha MSE**

In [23]:
np.linalg.norm(alpha - alphaNew)

1.4983354358871279

**Beta MSE**

In [24]:
np.linalg.norm(betaOld - betaNew)

0.76669992887715632

### *ii.	Comparison to Python package*

In addition to the generative model, the performance of our LDA implementation was compared to the implementation from the genism package. This implementation can be found at https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html. 

The results from the Python package and our LDA algorithm were not exactly the same for every topic. For each topic, three of the four words for each topic were the same and the probabilites were similar. However, the fourth word typically differed. This could be a result of some of the differences between the two algorithms. The Python package implmentation requires the user to pass the number of iterations as an argument whereas our implemnentation sets convergence criterion. In addition, the Python implementation randomly initiates all of the parameters. Thus, the differences between the starting positions and the convergence criterion could result in different topics. Therefore, we are satisfied with the similarity found between our LDA implementation and that using the gensim package.


In [25]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "The quick brown fox"
doc_b = "Brown fox jumps over the jumps jumps jumps"
doc_c =  "The the the lazy dog elephant."
doc_d = "The the the the the dog peacock lion tiger elephant"


# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, iterations = 1)

**Python Package Results**

In [26]:
print(ldamodel.print_topics(num_topics=3, num_words=4))

[(0, '0.192*jump + 0.124*brown + 0.106*fox + 0.101*dog'), (1, '0.187*jump + 0.120*fox + 0.117*eleph + 0.115*dog'), (2, '0.176*jump + 0.118*dog + 0.118*eleph + 0.109*fox')]


**Our LDA Function Results**

In [27]:
mostCommon(res[0][3], corpusMatrix[1], 4)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[              0
 jump   0.465856
 fox    0.181426
 dog    0.127051
 eleph  0.056676,               1
 brown  0.235505
 lazi   0.123889
 tiger  0.118565
 jump   0.103017,                 2
 eleph    0.217727
 dog      0.170284
 peacock  0.118463
 jump     0.098728]

### e.	Profiling Code and Speed Up

We profiled the initial code by implementing the algorithm on a corpus of all of President Clinton’s State of the Union Addresses. The profiling results can be found in the Appendix, Section A.2. It was found that the main bottleneck in the algorithm was the maximization step of the algorithm; about 75% of the time of the LDA algorithm was spent on this step. In the maximization step, the $\alpha$ update occurs in a while loop, and the $\beta$ update uses four for loops. The expectation step also slowed down the code, but only accounted for approximately 13% of the time of the LDA implementation. This mostly likely is a result of having fewer for loops than the maximization step. 

We identified inefficiencies in the matrix construction function and implemented a storage system in order to avoid the unnecessary repetition of some calculations. In addition, during the initial coding of the algorithm, care was taken to avoid for loops and use vectorization where possible. Just in time compilation (Numba) was also added to the code for the algorithm implementation. 

Given the difficulty of the algorithm and its dependence on previous iterations, it was not clear how parallelization would speed up the implmentation. Thus with the assistance of Professor Chan, the code was adapted so that Cython could be used. It was first necessary to adjust the code to prevent ragged arrays because Cython does not easily work with sparse or ragged arrays. The phi and the corpus matrix both initially used ragged arrays. To fix this, the largest dimension of the matrices was found, and the rest of the matrices were filled in with zeros such that the dimensions matched. In addition, the digamma function from the Cython GSL package replaced that from the scipy package. The final change was using cdef to define the inputs to the functions and the variables used within. 



**Toy  Example Timing:**

| Method| Total Time [s] |
|-----------|----------|
|      No Speed-Up|    0.00436|
|      Numba JIT|    3.02|
|     Cython|    0.00416|


**Clinton SOTU Timing:**

| Method| Total Time  |
|-----------|----------|
|      No Speed-Up|    4 min, 54 sec|
|      Numba JIT|    4 min, 51 sec|
|     Cython|    1 min, 30 sec|




As the tables above show, the Cython provided the fastest implementation of the algorithm. For the smaller example, Numba performed the slowest and was only marginally faster in the State of the Union example than no optimization. According to the Numba documentation, there is compilation time associated with Numba. In addition, it would be faster if Numba was used in No Python mode. However, this was not possible given the use of the numpy package. For the State of the Union example, the Cython code provides approximately a three time speed up over the other implementations.  There were 10,640 words used in the State of the Union example. 


**Time LDA on 8 Clinton State of the Union Speeches**

In [28]:
import nltk
from nltk.corpus import state_union

#creating a diciontary of the state of the Union Addresses for Clinton
fileNames = state_union.fileids()
Clinton = fileNames[50:58]

ClintonSOTU = {}
for i in range(0, len(Clinton)):
    ClintonSOTU[Clinton[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(Clinton[i]))

In [29]:
%%time
np.random.seed(7)
ClintCorp = make_word_matrix(ClintonSOTU, 0)
Results = LDA(3, ClintCorp, 0.1)

CPU times: user 1min 38s, sys: 1.64 s, total: 1min 40s
Wall time: 1min 42s


## IV.	Results

### *i. Topic Analysis*

The LDA algorithm was used to determine the topics that occurred in famous documents and speeches from American history. These included the Gettysburg Address, the Declaration of Independence, Martin Luther King’s “I Have A Dream” speech and President John F. Kennedy’s inauguration address. The first implementation defined the number of topics to be three. 

**American Speech Topics: 3 Topics:**



| Topic | Potential Occurring Themes |
|-----------|----------|
|      0 |     events, causes, opinions, impel |
|       1    |     laws, nature, connected, rights, life, happiness    |
|        2   |      people, disolve, political, another, God, Creator     |

In order to visualize the topic assignments, the following excerpt from The Declaration of Independence is included, where each word is highlighted to correspond to the color of the topic under which it occurs with the highest probability. 

<span style="color:red">Topic 0</span> 
 <span style="color:blue">Topic 1</span> 
 <span style="color:green">Topic 2</span> 

When in the <span style="color:blue">Course</span>  of <span style="color:green">human</span> <span style="color:red">events</span> , it <span style="color:red">becomes</span> <span style="color:green">necessary</span> for <span style="color:green">one</span> <span style="color:green">people</span> to <span style="color:green">dissolve</span> the <span style="color:green">political</span> <span style="color:red">bands</span> 
which have <span style="color:blue">connected</span> them with <span style="color:green">another</span>, and to <span style="color:blue">assume</span>, <span style="color:blue">among</span> the <span style="color:green">Powers</span> of the <span style="color:green">earth</span> , the <span style="color:green">separate</span> and <span style="color:green">equal</span> 
<span style="color:blue">station</span> to which the <span style="color:blue">Laws</span> of <span style="color:blue">Nature</span> and of <span style="color:blue">Nature's</span> <span style="color:green">God</span> <span style="color:green">entitle</span> them, a <span style="color:red">decent</span> <span style="color:red">respect</span> to the <span style="color:red">opinions</span> of 
<span style="color:red">mankind</span> <span style="color:green">requires</span> that they should 
<span style="color:green">declare</span> the <span style="color:red">causes</span> which <span style="color:red">impel</span> them to the <span style="color:green">separation</span>.

We <span style="color:blue">hold</span> these <span style="color:green">truths</span> to be <span style="color:blue">self-evident</span>, that all <span style="color:blue">men</span> are <span style="color:red">created</span> <span style="color:green">equal</span>, that they are <span style="color:blue">endowed</span> by their 
<span style="color:green">Creator</span> with <span style="color:blue">certain unalienable Rights</span>, that <span style="color:blue">among</span> these are <span style="color:blue">Life</span>, <span style="color:green">Liberty</span>, and the <span style="color:red">pursuit</span> of <span style="color:blue">Happiness</span>.

The second implementation defined the number of topics to be 10.



**American Speech Topics: 10 Topics:**

| Topic | Most Probable Words |
|-----------|----------|
|      0 |    us, let, time, come, justice  |
|      1     |    will, let, state, people, time      |
|      2     |    nation, new, dream, negro, one      |
|      3     |     will, us, nation, freedom, new     |
|       4    |    freedom, will, nation, right, power      |
|        5   |    freedom, day, nation, can, every      |
|      6     |    us, freedom, people, today, right      |
|     7      |     let, will, freedom, power, justice     |
|     8      |      will, let, us, one, state    |
|     9      |    will, let, nation, power, people      |

<span style="color:red">Topic 0</span> 
<span style="color:blue">Topic 1</span> 
<span style="color:green">Topic 2</span> 
<span style="color:darkmagenta">Topic 3</span> 
<span style="color:orange">Topic 4</span> 
<span style="color:lightseagreen">Topic 5</span> 
<span style="color:darkblue">Topic 6</span> 
<span style="color:magenta">Topic 7</span> 
<span style="color:cyan">Topic 8</span> 
<span style="color:chartreuse">Topic 9</span> 

**Declaration of Independence: 10 Topics**

When in the <span style="color:chartreuse">Course</span> of <span style="color:lightseagreen">human</span> <span style="color:red">events</span>, it <span style="color:red">becomes</span> <span style="color:green">necessary</span> for <span style="color:blue">one</span> <span style="color:orange">people</span> to <span style="color:darkmagenta">dissolve</span> the <span style="color:magenta">political</span> <span style="color:cyan">bands</span> 
which have <span style="color:darkblue">connected</span> them with <span style="color:darkmagenta">another</span>, and to <span style="color:darkmagenta">assume</span>, <span style="color:blue">among</span> the <span style="color:chartreuse">Powers</span> of the <span style="color:chartreuse">earth</span>, the <span style="color:chartreuse">separate</span> and <span style="color:orange">equal</span> 
<span style="color:lightseagreen">station</span> to which the <span style="color:orange">Laws</span> of <span style="color:lightseagreen">Nature</span> and of <span style="color:lightseagreen">Nature's</span> <span style="color:darkmagenta">God</span> <span style="color:magenta">entitle</span> them, a <span style="color:lightseagreen">decent</span> <span style="color:green">respect</span> to the <span style="color:darkmagenta">opinions</span> of 
<span style="color:lightseagreen">mankind</span> <span style="color:magenta">requires</span> that they should 
<span style="color:lightseagreen">declare</span> the <span style="color:orange">causes</span> which <span style="color:lightseagreen">impel</span> them to the <span style="color:chartreuse">separation</span>.

We <span style="color:magenta">hold</span> these <span style="color:blue">truths</span> to be <span style="color:chartreuse">self-evident</span>, that all <span style="color:red">men</span> are <span style="color:chartreuse">created</span> <span style="color:orange">equal</span>, that they are <span style="color:blue">endowed</span> by their 
<span style="color:lightseagreen">Creator</span> with <span style="color:orange">certain</span> <span style="color:chartreuse">unalienable</span> <span style="color:lightseagreen">Rights</span>, that <span style="color:blue">among</span> these are <span style="color:red">Life</span>, <span style="color:darkblue">Liberty</span>, and the <span style="color:cyan">pursuit</span> of <span style="color:magenta">Happiness</span>.

**Gettysburg Address: 10 Topics**

It is for <span style="color:lightseagreen">us</span> the <span style="color:red">living</span>, <span style="color:orange">rather</span>, to be <span style="color:orange">dedicated</span> here to the <span style="color:orange">unfinished</span>
<span style="color:magenta">work</span> which they who <span style="color:cyan">fought</span> here have <span style="color:orange">thus</span> <span style="color:darkblue">far</span> so <span style="color:lightseagreen">nobly</span> <span style="color:lightseagreen">advanced</span>.
It is <span style="color:orange">rather</span> for <span style="color:lightseagreen">us</span> to be here <span style="color:orange">dedicated</span> to the <span style="color:red">great</span> <span style="color:lightseagreen">task</span> <span style="color:chartreuse">remaining</span>
before <span style="color:lightseagreen">us</span>. . .that from these <span style="color:red">honored</span> <span style="color:darkblue">dead</span> we <span style="color:magenta">take</span> <span style="color:orange">increased</span> <span style="color:green">devotion</span>
to that <span style="color:orange">cause</span> for which they <span style="color:orange">gave</span> the <span style="color:orange">last</span> <span style="color:blue">full</span> <span style="color:magenta">measure</span> of <span style="color:green">devotion</span>. . .
that we here <span style="color:darkblue">highly</span> <span style="color:darkblue">resolve</span> that these <span style="color:darkblue">dead</span> <span style="color:magenta">shall</span> not have <span style="color:darkblue">died</span> in <span style="color:orange">vain</span>. . .
that this <span style="color:lightseagreen">nation</span>, under <span style="color:darkmagenta">God</span>, <span style="color:magenta">shall</span> have a <span style="color:darkblue">new</span> <span style="color:lightseagreen">birth</span> of <span style="color:cyan">freedom</span>. . .
and that <span style="color:darkblue">government</span> of the <span style="color:orange">people</span>. . .by the <span style="color:orange">people</span>. . .for the <span style="color:orange">people</span>. . .
<span style="color:magenta">shall</span> not <span style="color:lightseagreen">perish</span> from this <span style="color:chartreuse">earth</span>.

**Summary** 

For both the 3 topic and 10 topic LDA implmentations, the topic assignments make some sense, though to improve results, common words among all documents in the corpus should be removed or down-weighted and perhaps smoothing of the multinomial parameters should be performed as suggested in Section 5.4 of the orginial LDA paper (Blei et. al., 1006).

For 3 topics, words like happiness, rights, life and endowed are assigned the highest probability of being in the same topic and words like creator, God, power and liberty are also assigned to the same topic.  For 10 topics, however, it appears that there may be too many topics.  Creator and God are no long assigned to the same topic and neither is dead and perish.  However, the addition of more topics does allow for words like Nature, Creator and Mankind to be grouped together.  Likely somewhere between 3 and 10 topics would be optimal for this corpus.

In [30]:
file = open('Gettysburg.txt', 'r')
Gettysburg = file.read()
file2 = open("Dec_of_Independence.txt", 'r')
Declaration = file2.read()
file3 = open("MLK_Dream.txt", 'r')
MLK = file3.read()
file4 = open("JFK_Inauguration.txt", 'r')
JFK = file4.read()

speechDocs = collections.OrderedDict()
speechDocs["s1"] = Gettysburg
speechDocs["s2"] = Declaration
speechDocs["s3"] = MLK
speechDocs["s4"] = JFK

speechCorp = make_word_matrix(speechDocs, 1)

In [31]:
np.random.seed(17)
ResultsSpeech = LDA(3, speechCorp, 0.1)
ResultsSpeech2 = LDA(10, speechCorp, 0.1)

In [32]:
mostCommon(ResultsSpeech2[0][3], speechCorp[1], 5)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[               0
 us      0.017606
 let     0.011493
 time    0.011116
 come    0.010975
 justic  0.010577,               1
 will   0.019539
 let    0.015517
 state  0.013654
 peopl  0.013193
 time   0.011894,                2
 nation  0.019027
 new     0.014987
 dream   0.014013
 negro   0.012757
 one     0.012178,                 3
 will     0.023211
 us       0.021747
 nation   0.014964
 freedom  0.014338
 new      0.014234,                 4
 freedom  0.019594
 will     0.019362
 nation   0.018079
 right    0.013548
 power    0.012865,                 5
 freedom  0.015909
 day      0.012938
 nation   0.012780
 can      0.011266
 everi    0.010557,                 6
 us       0.027105
 freedom  0.018973
 peopl    0.014059
 today    0.012804
 right    0.011288,                 7
 let      0.024760
 will     0.023571
 freedom  0.017365
 power    0.011775
 justic   0.010406,               8
 will   0.025153
 let    0.024609
 us     0.020811
 one    0.012393
 state  0.011591,          

**Function to Assign Words to the Topic in which they Occur with the Highest Probability**

**colorCode(beta, wordList, inputWords):**

**Input:**
- *beta*: the output for beta from the LDA function, a k x V matrix
- *wordList*:the list of the unique words in the vocabulary in the corpus that LDA was run on to find beta, the output of the make_word_matrix()[1]
- *inputWords*: a sample of text from the corpus to assign each word to a topic

**Output: pd.DataFrame(topicWords)**
- *topicWords*: a pandas Data Frame where each row is a word from inputWords and the column is the topic under which that word occurs with the highest probability

In [33]:
def colorCode(beta, wordList, inputWords):
    
    #Split text, convert to lowercase and remove punctuation and stop words
    stopWords = get_stop_words('english')
    table = dict.fromkeys(map(ord, string.punctuation))
    text = inputWords.translate(table)
    words = text.lower().split()
    words = [word for word in words if word not in stopWords]
    p_stemmer = PorterStemmer()
    words = [p_stemmer.stem(word) for word in words]
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    
    
    k = len(words)
    topicWords = np.zeros((1,k))
    topicWords = pd.DataFrame(topicWords,  columns=words)

    for i in range(0, k):
        document = np.array(betaDF.loc[:,words[i]])
        topicWords.iloc[:,i] = np.argmax(document)
    return(topicWords)

In [34]:
GSample = """It is for us the living, rather, to be dedicated here to the unfinished
work which they who fought here have thus far so nobly advanced.
It is rather for us to be here dedicated to the great task remaining
before us. . .that from these honored dead we take increased devotion
to that cause for which they gave the last full measure of devotion. . .
that we here highly resolve that these dead shall not have died in vain. . .
that this nation, under God, shall have a new birth of freedom. . .
and that government of the people. . .by the people. . .for the people. . .
shall not perish from this earth."""

In [35]:
colors = colorCode(ResultsSpeech2[0][3], speechCorp[1], GSample).T

### *ii. Movie Data*

In addition to the topic creation for the historical documents, the LDA algorithm was used to cluster movies based on user ratings. This example used the MovieLens data set from the GroupLens website (http://grouplens.org/datasets/movielens/).  This data contains user ratings for 2,714 unique movies, and we use the data to determine clusters of movies based on which movies the users prefer. A movie is preferred if a user rated it four or five (out of five). The data was then reduced to only the information about users who had at least 50 preferred movies. In this example, the users can be thought of as the documents and their preferred movies are the words. The corpus is the collection of users. LDA will create topics for these movies that should cluster movies together based on which user preferred which movies. The LDA algorithm was used to create five clusters of movies. If a movie appeared in more than one cluster, it was only assigned to the cluster for which it had the highest probability.



In [36]:
#reading in the data
import pandas as pd
Ratings = pd.read_csv("RatingsDataClean.csv")
movieData = pd.read_csv("MovieReferenceTable.csv", encoding='cp1252')


#making a dictionary for the training and testing data
Users = list(set(Ratings['userID']))
RatingsDict = {}
for i in range(0, len(Users)):
    movies = list(map(str, list(pd.DataFrame(Ratings[Ratings['userID'] == Users[i]])['movieID'])))
    RatingsDict[i] = ' '.join(movies)

In [37]:
#running LDA analysis to get parameters for movie training data
import random

random.seed(77)
movieMatrix = make_word_matrix(RatingsDict, 1)
movieOutput = LDA(5, movieMatrix, 0.1)

In [38]:
movieTopics = mostCommon(movieOutput[0][3], movieMatrix[1], 15)
movieList = []
for i in range(0,5):
    movieList.append(list(np.array(movieTopics[i].index, dtype = int)))

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


In [39]:
betaDF = movieOutput[0][3]

k = len(movieMatrix[1])
topicWords = np.zeros((1,k))
topicWords = pd.DataFrame(topicWords,  columns=movieMatrix[1])

for i in range(0, k):
    document = np.array(betaDF[:,i])
    topicWords.iloc[:,i] = np.argmax(document)

topicWords = topicWords.T

topicWords['movieID'] = topicWords.index.astype(int)

movieData = topicWords.merge(movieData, on = "movieID", how = "left")
movieData.columns = ['cluster', 'movieID', 'UserID', "Name", "Genre"]


**Cluster 1**

The first cluster appears to be mostly composed of children's and family movies. One exception is Freeway, an R-rated  comedy, crime and thriller movie. In addition, Golden Eye is a James Bond movie that does not seem to very similar to the other movies in this cluster. Freeway and Golden Eye are probably movies enjoyed by similar viewers. So it is possible that this cluster is the reuslt of parents who watch a lot of children's movies but also enjoy action movies. 

In [40]:
#Cluster 1
movieData[movieData['cluster'] == 0].head(10)

,cluster,movieID,UserID,Name,Genre
0,0,1,0,Toy Story (1995),Animation|Children's|Comedy
6,0,1007,994,"Apple Dumpling Gang, The (1975)",Children's|Comedy|Western
18,0,1018,1005,That Darn Cat! (1965),Children's|Comedy|Mystery
39,0,1041,1028,Secrets & Lies (1996),Drama
45,0,1049,1035,"Ghost and the Darkness, The (1996)",Action|Adventure
59,0,1066,1052,Shall We Dance? (1937),Comedy|Musical|Romance
60,0,1067,1053,"Damsel in Distress, A (1937)",Comedy|Musical|Romance
64,0,1073,1058,Willy Wonka and the Chocolate Factory (1971),Adventure|Children's|Comedy|Fantasy
69,0,1080,1064,Monty Python's Life of Brian (1979),Comedy
77,0,1089,1073,Reservoir Dogs (1992),Crime|Thriller


**Cluster 2**

This cluster has fewer children's movies than Cluster 1. Of the non-children's movies, the remaining movies are mostly drama or thriller.  

In [41]:
#Cluster 2
movieData[movieData['cluster'] == 1].head(10)

,cluster,movieID,UserID,Name,Genre
1,1,10,9,GoldenEye (1995),Action|Adventure|Thriller
2,1,100,98,City Hall (1996),Drama|Thriller
7,1,1008,995,"Davy Crockett, King of the Wild Frontier (1955)",Western
8,1,1009,996,Escape to Witch Mountain (1975),Adventure|Children's|Fantasy
9,1,101,99,Bottle Rocket (1996),Comedy
11,1,1011,998,Herbie Rides Again (1974),Adventure|Children's|Comedy
26,1,1027,1014,Robin Hood: Prince of Thieves (1991): Drama: ...,Drama
30,1,1031,1018,Bedknobs and Broomsticks (1971),Adventure|Children's|Musical
33,1,1034,1021,Freeway (1996),Crime
37,1,1039,1026,Synthetic Pleasures (1995),Documentary


**Cluster 3**

This cluster also consistents of many children's movies. The movies that are not children's movies are the comedies Ed's Next Move, Bottle Rocket and That Thing You Do! This could be another instance of parents rating movies,  but instead of action movies they enjoy watching comedies when their children are not around. 

In [42]:
#Cluster 3
movieData[movieData['cluster'] == 2].head(10)

,cluster,movieID,UserID,Name,Genre
3,2,1002,989,Ed's Next Move (1996),Comedy
5,2,1005,992,D3: The Mighty Ducks (1996): Children's|Comed...,Children's|Comedy
14,2,1014,1001,Pollyanna (1960),Children's|Comedy|Drama
16,2,1016,1003,"Shaggy Dog, The (1959)",Children's|Comedy
17,2,1017,1004,Swiss Family Robinson (1960),Adventure|Children's
20,2,1020,1007,Cool Runnings (1993),Comedy
24,2,1024,1011,"Three Caballeros, The (1945)",Animation|Children's|Musical
28,2,1029,1016,Dumbo (1941),Animation|Children's|Musical
29,2,1030,1017,Pete's Dragon (1977),Adventure|Animation|Children's|Musical
32,2,1033,1020,"Fox and the Hound, The (1981)",Animation|Children's


**Cluster 4**

Cluster 4 had several musical children's movies and among the non-children's movies, contains many comedies and drama. 

In [43]:
#Cluster 4
movieData[movieData['cluster'] == 3].head(10)

,cluster,movieID,UserID,Name,Genre
4,3,1003,990,Extreme Measures (1996),Drama|Thriller
12,3,1012,999,Old Yeller (1957),Children's|Drama
22,3,1022,1009,Cinderella (1950),Animation|Children's|Musical
31,3,1032,1019,Alice in Wonderland (1951),Animation|Children's|Musical
34,3,1035,1022,"Sound of Music, The (1965)",Musical
38,3,104,102,Happy Gilmore (1996),Comedy
41,3,1043,1030,To Gillian on Her 37th Birthday (1996),Drama|Romance
43,3,1046,1033,Beautiful Thing (1996),Drama|Romance
47,3,1050,1036,Looking for Richard (1996),Documentary|Drama
50,3,1054,1040,Get on the Bus (1996),Drama


**Cluster 5**

Cluster 5 also has a lot of children's movies. The only movies that are not children's movies belong to the action/thriller genre. Again, these users may watch children's movies with their family, but prefer action movies when it is not family time. The children's movies are older, so it is possible that the family movie preferences have transitioned as the children grow up.

In [44]:
#Cluster 5
movieData[movieData['cluster'] == 4].head(10)

,cluster,movieID,UserID,Name,Genre
10,4,1010,997,"Love Bug, The (1969)",Children's|Comedy
13,4,1013,1000,"Parent Trap, The (1961)",Children's|Drama
15,4,1015,1002,Homeward Bound: The Incredible Journey (1993)...,Adventure|Children's
19,4,1019,1006,"20,000 Leagues Under the Sea (1954)",Adventure|Children's|Fantasy|Sci-Fi
21,4,1021,1008,Angels in the Outfield (1994),Children's|Comedy
23,4,1023,1010,Winnie the Pooh and the Blustery Day (1968),Animation|Children's
25,4,1025,1012,"Sword in the Stone, The (1963)",Animation|Children's
27,4,1028,1015,Mary Poppins (1964),Children's|Comedy|Musical
36,4,1037,1024,"Lawnmower Man, The (1992)",Action|Sci-Fi|Thriller
44,4,1047,1034,"Long Kiss Goodnight, The (1996)",Action|Thriller


## V.	Further Research

One area in which we would like to expand our research and improve our project is in the approximation of the posterior. In the implementation for this project, variational inference was utilized to estimate the intractable posterior distribution. However, there are other methods that can be used to estimate the posterior. These include Gibbs sampling and Metropolis-Hastings. It is possible that these approximations might increase the speed of the implementation. 
Another area to expand on is the determination of the number of topics. The R implementation of LDA provides different measures to determine the number of topics, and these are something that we can look into adding to our code.

In order to improve the topic definitions, we want to investigate weighting terms by the number of times that they appear in the documents. Many of the results  in this report contain topics that have the same words in each of them. It is sensical that the same word may appear in documents on similar topics, such as American historical documents, but if a word appears many times in each document, that word will not be very helpful in defining topics. Another way to improve topic definition would be to implement smoothing into the algorithm. If a word only occurs once in a corpus, it will have a very small probability. However, this word could be important in defining the topic for a particular document. Smoothing would ensure that every word would be assigned a positive probability. 

In addition, we want to continue to use the algorithms to situations beyond text corpora. The movie data example in this paper is one example of how LDA can be utilized beyond text data. The algorithm can be used for any situation that has the same structure as text corpora. The possibilities for LDA applications are endless, and we hope to explore more of these situations in the future. 


## VI. References

1.	Colorado Reed, Latent Dirichlet Allocation: Towards a Deeper Understanding, January 2012, o
bphio.us/pdfs/lda_tutorial.pdf. 
2.	David M. Blei, Andrew Y. Ng, and Michael I. Jordan, Latent Dirichlet Allocation, Journal of 
Machine Learning Research 3, 2003, pg. 993-1022.
3. Declaration of Independence: http://www.gutenberg.org/files/16780/16780-h/16780-h.html
4. F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872
5. Gettysburg Address: http://www.gutenberg.org/cache/epub/4/pg4.txt.
6. Internet Movie Database, www.imbd.com.
7. JFK Speech: http://www.americanrhetoric.com/speeches/jfkinaugural.htm
8. Max Sklar, Fast MLE Computation for the Dirichlet Multinomial, May 2014.
9. MLK Speech:  http://www.americanrhetoric.com/speeches/mlkihaveadream.htm
10. Thomas P. Minka, Estimating a Dirichlet Distribution, www.msr-waypoint.com.